In [1]:
import os
import sys
sys.path.append('./DreamDiffusion/code/')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from timm.models.vision_transformer import Block

c:\Users\kshch\Projects\Наука\Разработка\EEGtoImageDecoder\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
class TUABDataset(Dataset):
    def __init__(self, path, normal='/normal/01_tcp_ar', abnormal='/abnormal/01_tcp_ar'):
        self.normal_path = path + normal
        self.abnormal_path = path + abnormal
        self.file_paths = []
        self.labels = []

        for file_name in os.listdir(self.normal_path):
            file_path = self.normal_path + "/" + file_name
            self.file_paths.append(file_path)
            self.labels.append(0)

        for file_name in os.listdir(self.abnormal_path):
            file_path = self.abnormal_path + "/" + file_name
            self.file_paths.append(file_path)
            self.labels.append(1)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        if idx < 0:
            idx = len(self.file_paths) + idx
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        return (np.load(file_path, allow_pickle=True), label)

In [17]:
path = r"./datasets/tuab_resampled/tuh_eeg_abnormal/v3.0.1/edf/train"
dataset = TUABDataset(path)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

In [18]:
for batch, labels in dataloader:
    print(batch, labels)
    break

tensor([[[ 1.5077e-05,  1.8362e-05,  2.0422e-05,  ...,  2.5208e-05,
           2.6685e-05,  2.6369e-05],
         [ 6.8156e-06,  1.5318e-05,  2.1212e-05,  ...,  2.5380e-05,
           3.0066e-05,  3.2898e-05],
         [-2.8378e-07,  2.3607e-07,  7.6191e-07,  ...,  3.3891e-06,
           3.2079e-06,  2.3966e-06],
         ...,
         [-8.5057e-06, -8.5282e-06, -8.3740e-06,  ..., -8.6897e-06,
          -8.9864e-06, -9.2011e-06],
         [-8.7620e-06, -7.7773e-06, -7.9433e-06,  ..., -2.1111e-05,
          -1.8660e-05, -1.7707e-05],
         [-1.1352e-05, -8.1800e-06, -6.9073e-06,  ..., -1.6525e-05,
          -1.3627e-05, -1.1509e-05]],

        [[ 1.7200e-05,  1.6878e-05,  1.5956e-05,  ...,  6.9065e-06,
           7.7190e-06,  8.8457e-06],
         [ 2.5213e-05,  3.1748e-05,  3.7053e-05,  ...,  9.0471e-06,
           6.6556e-06,  3.1777e-06],
         [ 3.8714e-07, -1.0228e-06, -2.1720e-06,  ..., -1.2306e-05,
          -1.0530e-05, -7.7625e-06],
         ...,
         [-1.4586e-05, -1